# 朴素贝叶斯

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math

In [2]:
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target
    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    data = np.array(df.iloc[:100, :])
    # print(data)
    return data[:,:-1], data[:,-1]

In [3]:
X, y = create_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
# 高斯朴素贝叶斯，使用高斯分布求解条件概率
class GaussianNaiveBayes():
    def __init__(self):
        self.model = None
        
    # 计算mean
    @staticmethod
    def mean(X):
        return sum(X) / float(len(X))
    
    def stdev(self, X):
        avg = self.mean(X)
        return math.sqrt(sum([pow(x - avg,2) for x in X]) / float(len(X)))
    
    # 高斯分布概率密度函数
    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(pow(x-mean,2) / (2 * pow(stdev, 2))))
        return (1 / (math.sqrt(2 * math.pi ) * stdev) * exponent)
    
    # 计算每一列数据的均值和方差，数据独立同分布,zip(*解压)
    def summarize(self, train_data):
        summaries = [(self.mean(x), self.stdev(x)) for x in zip(*train_data)]
        return summaries
    
    #对数据按类别分类，之后求出数学期望和标准差
    def fit(self, X, y):
        labels = list(set(y))
        data = {label:[] for label in labels}
        for f ,label in zip(X,y):
            data[label].append(f)
        # 计算出每个label和对应的数据的mean和方差
        self.model = {label: self.summarize(value) for label, value in data.items()}
        return 'GaussianNB train done'
    
    def calculate_probabilities(self, input_data):
        # label-value  summaries:{0.0: [(5.0, 0.37),(3.42, 0.40)], 1.0: [(5.8, 0.449),(2.7, 0.27)]}
        # input_data:[1.1, 2.2]
        probabilities = {}
        # 遍历所有数据，每条数据计算一次
        for label, value in self.model.items():
            probabilities[label] = 1
            for i in range(len(value)):
                mean,stdev = value[i]
                probabilities[label] *= self.gaussian_probability(input_data[i],mean, stdev)
        return probabilities        
    
    def predict(self, X_test):
        # {0.0: 2.9680340789325763e-27, 1.0: 3.5749783019849535e-26}
        label = sorted(self.calculate_probabilities(X_test).items(), key=lambda x: x[-1])[-1][0]
        return label
    
    def score(self, X_test, y_test):
        right = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right += 1

        return right / float(len(X_test))

In [8]:
model = GaussianNaiveBayes()

In [9]:
model.fit(X_train, y_train)

<generator object GaussianNaiveBayes.summarize.<locals>.<genexpr> at 0x7fdca11f9de0>
<generator object GaussianNaiveBayes.summarize.<locals>.<genexpr> at 0x7fdca11f9de0>


'GaussianNB train done'

In [15]:
print(model.predict([4.4,  3.2,  1.3,  0.2]))

0.0


In [16]:
model.score(X_test, y_test)

1.0